In [1]:
!pip install dash plotly scikit-learn tensorflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 21.1 MB/s eta 0:00:00


In [2]:
# Install requirements
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization

from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Dense
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Concatenate, Input
from tensorflow.keras.models import Model
import plotly.graph_objects as go
from dash.dependencies import Input as DashInput, Output as DashOutput
import dash
from dash import dcc, html, Input, Output


In [3]:
# -------------------------------
# CONFIGURATION
# -------------------------------
n_locations = 5
seq_len = 30
future_steps = 10
data_store = pd.DataFrame()
model, scaler, iso_forest = None, None, None

# Generate static (x, y, z) coordinates for mine sensors
coords_df = pd.DataFrame({
    'location_id': [f"Loc-{i}" for i in range(n_locations)],
    'x': np.random.randint(0, 100, n_locations),
    'y': np.random.randint(0, 100, n_locations),
    'z': -np.random.randint(20, 100, n_locations)  # underground depth
})


In [32]:
# -------------------------------
# 1. Simulate Sensor Data
# -------------------------------
def simulate_data():
    global data_store
    timestamp = datetime.now()
    records = []
    for _, row in coords_df.iterrows():
        records.append({
            'timestamp': timestamp,
            'location_id': row['location_id'],
            'x': row['x'],
            'y': row['y'],
            'z': row['z'],
            'methane': np.random.normal(0.7, 0.2),
            'temperature': np.random.normal(28, 1),
            'humidity': np.random.normal(65, 4),
            'vibration': np.random.normal(0.2, 0.05),
            'pressure': np.random.normal(1010, 15)
        })
    df = pd.DataFrame(records)
    data_store = pd.concat([data_store, df], ignore_index=True)
    data_store = data_store.groupby('location_id').tail(200).reset_index(drop=True)


In [33]:
def build_robust_lstm(seq_len):
    model = Sequential([
        LSTM(64, return_sequences=True, activation='relu', input_shape=(seq_len, 1)),
        BatchNormalization(),
        Dropout(0.3),

        LSTM(64, return_sequences=True, activation='relu'),
        BatchNormalization(),
        Dropout(0.25),

        LSTM(32, activation='relu'),
        Dropout(0.2),

        Dense(64, activation='relu'),
        Dropout(0.15),

        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [34]:

# -------------------------------
# 2. Train Model
# -------------------------------
def train_model():
    global model, scaler, iso_forest

    df = data_store[data_store['location_id'] == coords_df['location_id'].iloc[0]].copy()
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df[['methane']])

    X, y = [], []
    for i in range(len(scaled) - seq_len):
        X.append(scaled[i:i+seq_len])
        y.append(scaled[i+seq_len])

    X = np.array(X)
    y = np.array(y)

    # Reshape X for LSTM: [samples, time_steps, features]
    X = X.reshape((X.shape[0], X.shape[1], 1))

    model = build_robust_lstm(seq_len)

    # Train the model
    model.fit(X, y, epochs=10, batch_size=32, verbose=0)

    # Train Isolation Forest for anomaly detection
    features = ['methane', 'temperature', 'humidity', 'vibration', 'pressure']
    iso_forest = IsolationForest(contamination=0.05)
    iso_forest.fit(df[features].values)



In [35]:
# -------------------------------
# 3. Forecast Methane (toxic gas)
# -------------------------------
def forecast():
    df = data_store[data_store['location_id'] == coords_df['location_id'].iloc[0]]
    seq = scaler.transform(df[['methane']])[-seq_len:]
    seq = seq.reshape(1, seq_len, 1)
    preds = []
    for _ in range(future_steps):
        pred = model.predict(seq, verbose=0)[0][0]
        preds.append(pred)
        seq = np.append(seq[:, 1:, :], [[[pred]]], axis=1)
    return scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()


In [36]:
# -------------------------------
# DASH APP
# -------------------------------
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H2(" AI-Powered Mine Monitoring prototype with Dynamic 3D Sensor Locations"),
    dcc.Interval(id='interval', interval=3000, n_intervals=0),
    dcc.Graph(id='graph-3d'),
    html.Div(id='alerts', style={'color': 'red', 'whiteSpace': 'pre-line', 'marginTop': '20px'})
])

@app.callback(
    [DashOutput('graph-3d', 'figure'), DashOutput('alerts', 'children')],
    [DashInput('interval', 'n_intervals')]
)
def update_dashboard(n):
    simulate_data()
    if n == 2 or n % 20 == 0:
        train_model()

    preds = forecast()
    latest = data_store.groupby('location_id').tail(1)

    # Build 3D visualization
    fig = go.Figure(go.Scatter3d(
        x=latest['x'],
        y=latest['y'],
        z=latest['z'],
        mode='markers',
        marker=dict(
            size=12,
            color=latest['methane'],
            colorscale='Reds',
            colorbar=dict(title='Methane (%)'),
            cmin=0,
            cmax=1.5
        ),
        hovertext=[
            f"Sensor: {row['location_id']}<br>"
            f"Coords: ({row['x']}, {row['y']}, {row['z']})<br>"
            f"Methane: {row['methane']:.2f} %<br>"
            f"Temp: {row['temperature']:.1f} °C<br>"
            f"Humidity: {row['humidity']:.1f} %<br>"
            f"Vibration: {row['vibration']:.3f}<br>"
            f"Pressure: {row['pressure']:.1f} kPa"
            for _, row in latest.iterrows()
        ],
        hoverinfo='text'
    ))

    fig.update_layout(
        title="3D Mine Sensor Network - Methane Risk Map",
        scene=dict(
            xaxis_title='X (meters)',
            yaxis_title='Y (meters)',
            zaxis_title='Depth (meters)',
            zaxis=dict(range=[-120, 0])
        ),
        margin=dict(l=0, r=0, t=40, b=0)
    )

    # Generate dynamic alert messages
    alerts = []
    affected_zones = []

    for _, row in latest.iterrows():
        loc = row['location_id']
        x, y, z = row['x'], row['y'], row['z']
        methane = row['methane']
        if methane > 1.5:
            alerts.append(
                f"⚠️ CRITICAL: Methane = {methane:.2f}% at {loc} (x={x}, y={y}, z={z})"
            )
            affected_zones.append(f"{loc} (x={x}, y={y}, z={z})")
        elif methane > 1.0:
            alerts.append(
                f"⚠️ WARNING: Methane = {methane:.2f}% at {loc} (x={x}, y={y}, z={z})"
            )
            affected_zones.append(f"{loc} (x={x}, y={y}, z={z})")

    # Append a dynamic ART message if any affected
    if alerts:
        alerts.append(
            "\n🎨 ART MESSAGE:\n"
            f"Hazardous methane detected at:\n  - " + "\n  - ".join(affected_zones) +
            "\n🚨 Immediate response required for these zones!"
        )
        return fig, "\n".join(alerts)

    return fig, "✅ All zones are safe. No methane warnings or critical levels."


In [9]:
simulate_data()
simulate_data()
app.run(debug=False)

<IPython.core.display.Javascript object>